In [1]:
import pandas as pd
import numpy as np
import os
import glob
import sqlite3
import gc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unzip /content/drive/MyDrive/Projeto_Final_BI/Mortalidade_Geral.zip -d /content/drive/MyDrive/Projeto_Final_BI/

In [3]:
URL = "/content/drive/MyDrive/Projeto_Final_BI/"

In [23]:
conn = sqlite3.connect(URL+"staging.db")

In [5]:
df_18 = pd.read_csv(URL+"Mortalidade_Geral_2018.csv", sep=";")

/tmp/ipython-input-1332851786.py:1: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df_18 = pd.read_csv(URL+"Mortalidade_Geral_2018.csv", sep=";")


In [6]:
df_19 = pd.read_csv(URL+"Mortalidade_Geral_2019.csv", sep=";")

/tmp/ipython-input-103563467.py:1: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df_19 = pd.read_csv(URL+"Mortalidade_Geral_2019.csv", sep=";")


In [7]:
df_20 = pd.read_csv(URL+"Mortalidade_Geral_2020.csv", sep=";")

/tmp/ipython-input-4102249514.py:1: DtypeWarning: Columns (66,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df_20 = pd.read_csv(URL+"Mortalidade_Geral_2020.csv", sep=";")


In [8]:
df_21 = pd.read_csv(URL+"Mortalidade_Geral_2021.csv", sep=";")

/tmp/ipython-input-1270836982.py:1: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df_21 = pd.read_csv(URL+"Mortalidade_Geral_2021.csv", sep=";")


In [9]:
df_22 = pd.read_csv(URL+"Mortalidade_Geral_2022.csv", sep=";")

/tmp/ipython-input-794182228.py:1: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df_22 = pd.read_csv(URL+"Mortalidade_Geral_2022.csv", sep=";")


In [10]:
df_23 = pd.read_csv(URL+"Mortalidade_Geral_2023.csv", sep=";")
df_23 = df_23.rename(columns={'contador': 'CONTADOR'})

/tmp/ipython-input-3465513044.py:1: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  df_23 = pd.read_csv(URL+"Mortalidade_Geral_2023.csv", sep=";")


In [11]:
df_24 = pd.read_csv(URL+"Mortalidade_Geral_2024.csv", sep=";")
df_24 = df_24.rename(columns={'contador': 'CONTADOR'})

/tmp/ipython-input-1797566191.py:1: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  df_24 = pd.read_csv(URL+"Mortalidade_Geral_2024.csv", sep=";")


In [12]:
cols_18 = df_18.columns.values
cols_23 = df_23.columns.values
cols_24 = df_24.columns.values

set18 = set(cols_18)
set23 = set(cols_23)
set24 = set(cols_24)

In [13]:
colunas_18 = list(set18 - set24)
colunas_24 = list(set24 - set18)

to_drop = ["HORAOBITO", "CODMUNNATU", "ESC", "SERIESCFAL", "OCUP",
           "LOCOCOR", "CODESTAB", "ESTABDESCR", "IDADEMAE", "ESCMAE",
           "ESCMAE2010", "SERIESCMAE", "OCUPMAE", "QTDFILVIVO", "QTDFILMORT", "SEMAGESTAC", "CB_PRE",
           "DTATESTADO", "FONTE", "NUMEROLOTE", "DTINVESTIG", "CAUSABAS_O",
           "DTCADASTRO", "ATESTANTE", "STCODIFICA", "CODIFICADO", "VERSAOSIST",
           "VERSAOSCB", "FONTEINV", "DTRECEBIM", "ESCMAEAGR1", "ESCFALAGR1",
           "STDONOVA", "NUDIASOBCO", "NUDIASOBIN", "DTCADINV", "DTCONINV",
           "FONTES", "TPNIVELINV", "NUDIASINF", "DTCADINF", "DTCONCASO",
           "FONTESINF"]

lista_to_drop = to_drop + colunas_18 + colunas_24

lista_to_drop = list(set(lista_to_drop))

In [14]:
df_keys = {
    "2018" : df_18,
    "2019" : df_19,
    "2020" : df_20,
    "2021" : df_21,
    "2022" : df_22,
    "2023" : df_23,
    "2024" : df_24
}

In [19]:
for key in list(df_keys.keys()):
  df_keys[key].drop(columns=[c for c in df_keys[key].columns if c in lista_to_drop],
                      inplace=True)

In [24]:
for key in list(df_keys.keys()):
    df_keys[key].to_sql("mortalidade", conn, if_exists="append", index=False)
    print(f"Tabela mortalidade_{key} criada com sucesso! Com {len(df_keys[key])} registros.")
    del df_keys[key]
    gc.collect()

Tabela mortalidade_2018 criada com sucesso! Com 1316719 registros.
Tabela mortalidade_2019 criada com sucesso! Com 1349801 registros.
Tabela mortalidade_2020 criada com sucesso! Com 1556824 registros.
Tabela mortalidade_2021 criada com sucesso! Com 1832649 registros.
Tabela mortalidade_2022 criada com sucesso! Com 1544266 registros.
Tabela mortalidade_2023 criada com sucesso! Com 1465610 registros.
Tabela mortalidade_2024 criada com sucesso! Com 1426346 registros.


In [26]:
cursor = conn.cursor()

In [27]:
#Nome de todas as tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = [t[0] for t in cursor.fetchall()]
print("Tabelas:", tabelas)

Tabelas: ['mortalidade']


In [28]:
colunas_por_tabela = {}

for tabela in tabelas:
    cursor.execute(f'PRAGMA table_info("{tabela}")')
    colunas = [row[1].upper() for row in cursor.fetchall()]  # pega os nomes em maiúsculo
    colunas_por_tabela[tabela] = colunas


In [29]:
for item in colunas_por_tabela.values():
  print(item)

['ORIGEM', 'TIPOBITO', 'DTOBITO', 'NATURAL', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC2010', 'CODMUNRES', 'CODMUNOCOR', 'GRAVIDEZ', 'GESTACAO', 'PARTO', 'OBITOPARTO', 'PESO', 'TPMORTEOCO', 'OBITOGRAV', 'OBITOPUERP', 'ASSISTMED', 'EXAME', 'CIRURGIA', 'NECROPSIA', 'LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'CAUSABAS', 'COMUNSVOIM', 'CIRCOBITO', 'ACIDTRAB', 'TPPOS', 'ATESTADO', 'DTRECORIGA', 'CAUSAMAT', 'STDOEPIDEM', 'DIFDATA', 'TPOBITOCOR', 'TPRESGINFO', 'MORTEPARTO', 'ALTCAUSA', 'CONTADOR']


In [30]:
cursor.execute("SELECT COUNT(CONTADOR) FROM mortalidade")
cursor.fetchall()

[(10492215,)]

In [31]:
df_cid = pd.read_csv(URL+"CID_10.csv", sep=";")
df_cid.to_sql("cid", conn, if_exists="replace", index=False)

14233

In [33]:
df_obitos_0_4 = pd.read_csv(URL+"Obitos_Evitaveis_0_a_4.csv", sep=";")
df_obitos_5_74 = pd.read_csv(URL+"Obitos_Evitaveis_5_a_74.csv", sep=";")

df_obitos_0_4.to_sql("obitos_evitaveis_infantil", conn, if_exists="replace", index=False)
df_obitos_5_74.to_sql("obitos_evitaveis_adulto", conn, if_exists="replace", index=False)

4564